In [52]:

admin_pk = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"

unit_pk1 = "0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d"

employee_pk1 = "0x7c852118294e51e653712a81e05800f419141751be58f605c371e15141b007a6"
employee_pk2 = "0x47e179ec197488593b187f80a00eb0da91f1b9d0b13f8733639f19c30a34926a"

In [83]:
endpoint_uri="http://127.0.0.1:8545"
import json
from web3 import Web3


# 配置 web3 contract 实例
taxmgr_contract_address = "0x5FC8d32690cc91D4c39d9d3abcBD16989F875707"
rmbtk_contract_address = "0xCf7Ed3AccA5a467e9e704C703E8D87F634fB0Fc9"
taxtk_contract_address = "0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9"

with open('../contract/out/ERC20.sol/ERC20.json', 'r') as f:
    erc20_json = json.load(f)
    erc20_abi = erc20_json['abi']

with open('../contract/out/TaxManagement.sol/TaxManagement.json', 'r') as f:
    taxmgr_json = json.load(f)
    taxmgr_abi = taxmgr_json['abi']

web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))


contract = web3.eth.contract(address=taxmgr_contract_address, abi=taxmgr_abi)
rmbtk_contract = web3.eth.contract(address=rmbtk_contract_address, abi=erc20_abi)
taxtk_contract = web3.eth.contract(address=taxtk_contract_address, abi=erc20_abi)


In [42]:
def submit_txn(private_key, transaction):
    # 签名交易
    signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
    print("signed_txn: ", signed_txn)

    # 发送交易
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print("tx_hash: ", tx_hash)

    # 等待交易完成
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("receipt: ", receipt)

    return receipt

In [10]:

def get_salary_records_from_blockchain(private_key):

    # 通过私钥得到地址
    PA=web3.eth.account.from_key(private_key)
    account = PA.address
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    print(dir(contract.events.TaxCalculated))

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.TaxCalculated.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'employeeAddress': account}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()
    print("Events: ", events)

    # 解析事件日志
    salary_data = []
    for event in events:
        print("Event: ", event)
        salary_data.append({
            "salary_date": event['args']['timestamp'],  # 假设事件中有一个时间戳字段
            "gross_salary": event['args']['grossSalary'],  # 假设事件中有税前工资字段
            "net_salary": event['args']['netSalary'],
            "tax_amount": event['args']['taxAmount'],
        })

    print("Salary Data: ", salary_data)

    return salary_data


print(get_salary_records_from_blockchain("0x7c852118294e51e653712a81e05800f419141751be58f605c371e15141b007a6"))

Account:  0x90F79bf6EB2c4f870365E785982E1f101E93b906
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_event_abi', '_get_event_filter_params', '_parse_logs', '_process_get_logs_argument_filters', '_set_up_filter_builder', 'abi', 'address', 'build_filter', 'check_for_forbidden_api_filter_arguments', 'contract_abi', 'create_filter', 'event_name', 'factory', 'get_logs', 'process_log', 'process_receipt', 'w3']
Events:  []
Salary Data:  []
[]


In [26]:
def add_unit(private_key, unit_address):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.addUnit(unit_address).build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    # 签名交易
    signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
    print("signed_txn: ", signed_txn)

    # 发送交易
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print("tx_hash: ", tx_hash)

    # 等待交易完成
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("receipt: ", receipt)
    return receipt

unit_address = web3.eth.account.from_key(unit_pk1).address

add_unit(admin_pk, unit_address)

ContractLogicError: ('execution reverted: revert: Unit already exists', '0x08c379a000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000013556e697420616c72656164792065786973747300000000000000000000000000')

In [37]:
def get_units(private_key):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.UnitAdded.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])

get_units(admin_pk)

Account:  <eth_account.signers.local.LocalAccount object at 0x7f301bca4640>
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8'}), 'event': 'UnitAdded', 'logIndex': 1, 'transactionIndex': 0, 'transactionHash': HexBytes('0x4d2387f1bd04294170b53fcbaa26c8d2bdaf9a2e0e9bfd04ae1a8870c301f664'), 'address': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'blockHash': HexBytes('0x69f2cbb1345c6524398de41b2b395f4afcc255f3f40169891a502caa5ab16d49'), 'blockNumber': 4})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8


In [53]:
def add_employee(uni_pk, employee_pk, month_salary):

    account =  web3.eth.account.from_key(uni_pk)

    employee_address = web3.eth.account.from_key(employee_pk).address

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.addEmployee(
        employee_address, month_salary).build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    submit_txn(uni_pk, transaction)


add_employee(unit_pk1, employee_pk2, 10000)

transaction:  {'value': 0, 'gas': 120082, 'maxFeePerGas': 2100625808, 'maxPriorityFeePerGas': 1000000000, 'chainId': 31337, 'from': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'nonce': 2, 'to': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'data': '0xe7fd9a1300000000000000000000000015d34aaf54267db7d7c367839aaf71a00a2c6a650000000000000000000000000000000000000000000000000000000000002710'}
signed_txn:  SignedTransaction(rawTransaction=HexBytes('0x02f8b2827a6902843b9aca00847d3501908301d512945fc8d32690cc91d4c39d9d3abcbd16989f87570780b844e7fd9a1300000000000000000000000015d34aaf54267db7d7c367839aaf71a00a2c6a650000000000000000000000000000000000000000000000000000000000002710c080a0e9b7e5bf5017b5fe240d47f6105d8a24a9e9e68c2877d095c288b72b174a95b8a02c18895a7c96dea0799895c2d01205c38fe738c99ee18caae13d41c6be405544'), hash=HexBytes('0xbe308ce5d113c5e7581e7b07f2d584171391b5a1a62561057b39fa96ea664bfa'), r=105713812390453478723920809151516819459588331376510160473940984239727684785592, s=199451176459839

In [58]:
def get_employee(private_key):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.EmployeeAdded.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'unitAddress': account.address}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])
        print("employee address:", event['args']['employeeAddress'])
        print()

get_employee(unit_pk1)

Account:  <eth_account.signers.local.LocalAccount object at 0x7f301b755a50>
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'employeeAddress': '0x90F79bf6EB2c4f870365E785982E1f101E93b906', 'monthlySalary': 100}), 'event': 'EmployeeAdded', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x15c960ff346ef79b6936a3ea35cf901adc190c3b58c4506e4b68b6587add79b1'), 'address': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'blockHash': HexBytes('0x14491400f459d06fc44c172011f8927e0ad761b4b841a9341ce4fe2ddd54d10b'), 'blockNumber': 5})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
employee address: 0x90F79bf6EB2c4f870365E785982E1f101E93b906

Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'employeeAddress': '0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65', 'monthlySalary': 10000}), 'event': 'EmployeeAdded', 'logIndex': 0, 'transactionIndex': 0, 'transactionHas

In [59]:
def settleTaxes(private_key):

    account =  web3.eth.account.from_key(private_key)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.settleTaxes().build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    submit_txn(private_key, transaction)


settleTaxes(unit_pk1)

transaction:  {'value': 0, 'gas': 195194, 'maxFeePerGas': 1963755010, 'maxPriorityFeePerGas': 1000000000, 'chainId': 31337, 'from': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'nonce': 3, 'to': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'data': '0xdfab5dc3'}
signed_txn:  SignedTransaction(rawTransaction=HexBytes('0x02f871827a6903843b9aca0084750c86028302fa7a945fc8d32690cc91d4c39d9d3abcbd16989f8757078084dfab5dc3c080a0c2dff6f5cc53d299279cd78961370de5fd1f240dafb98b3722d89c066c1a8950a06d9d94526026a05bfe3f4259cb2c5a3aff0973b53e933945da019a09bbee9501'), hash=HexBytes('0x72bd646bbef1f20ef17c8763fdce8bec9155f47d11b4e2aefdf64a2038681704'), r=88144403976905693118827661767647406331770101072757354474967925511594524182864, s=49580519164047064363243416317978885738451234189378757672115798021194023605505, v=0)
tx_hash:  b'r\xbddk\xbe\xf1\xf2\x0e\xf1|\x87c\xfd\xce\x8b\xec\x91U\xf4}\x11\xb4\xe2\xae\xfd\xf6J 8h\x17\x04'
receipt:  AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 195194, 'lo

In [61]:
# def getTaxes(private_key):

#     account =  web3.eth.account.from_key(private_key)

#     # 构建交易
#     nonce = web3.eth.get_transaction_count(account.address)
#     transaction = contract.functions.getTaxes().build_transaction({
#         'from': account.address,
#         'nonce': nonce,
#     })
#     print("transaction: ", transaction)

#     submit_txn(private_key, transaction)

def getTaxes(private_key):
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.TaxSettlement.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'unitAddress': account.address}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])
        print("totalTaxDue:", event['args']['totalTaxDue'])
        print()

getTaxes(unit_pk1)

Account:  <eth_account.signers.local.LocalAccount object at 0x7f301b487fd0>
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'totalTaxDue': 0}), 'event': 'TaxSettlement', 'logIndex': 2, 'transactionIndex': 0, 'transactionHash': HexBytes('0x8775b878dfb19b3b6c38aa0ad39984bef80682fc325f77d8b90bbaabdddcfc88'), 'address': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'blockHash': HexBytes('0xb67d6e7c8e48adb6b2b3fc25aa19c75ba1042fff5e28b6ff1a264b832733d3f1'), 'blockNumber': 6})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
totalTaxDue: 0

Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'totalTaxDue': 745}), 'event': 'TaxSettlement', 'logIndex': 4, 'transactionIndex': 0, 'transactionHash': HexBytes('0x72bd646bbef1f20ef17c8763fdce8bec9155f47d11b4e2aefdf64a2038681704'), 'address': '0x5FC8d32690cc91D4c39d9d3abcBD16989F875707', 'blockHash': HexBytes('0xf727c35c2103b51cf8

In [93]:
def payTaxes(private_key, amount):

    account =  web3.eth.account.from_key(private_key)
    print("account", account)
    print("account address", account.address)
    print("amount: ", amount)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    try:
        # rmbtk_contract.functions.approve(taxmgr_contract_address, amount).transact({'from': account.address})
        # txhash=taxtk_contract.functions.approve(taxmgr_contract_address, amount).transact({'from': account.address})
        # # print(rmbtk_contract.functions.allowance(taxmgr_contract_address, amount).call())
        # tx_receipt = web3.eth.wait_for_transaction_receipt(txhash)
        # print("tx_receipt:", tx_receipt)
        print(taxtk_contract.functions.allowance(account.address, taxmgr_contract_address).call())
        transaction = contract.functions.payTaxes(account.address, amount, amount).build_transaction({
            'from': account.address,
            'nonce': nonce,
        })
    except Exception as e:
        print("err:", e)
        return
    print("transaction: ", transaction)

    submit_txn(private_key, transaction)

payTaxes(unit_pk1, 745)

account <eth_account.signers.local.LocalAccount object at 0x7f301af20550>
account address 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
amount:  745
745


ContractCustomError: ('0xfb8f41b20000000000000000000000005fc8d32690cc91d4c39d9d3abcbd16989f875707000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e9', '0xfb8f41b20000000000000000000000005fc8d32690cc91d4c39d9d3abcbd16989f875707000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e9')